# ЭКСПЕРТНЫЕ МЕТОДЫ

## 1.1 Непосредственное назначение весовых коэф.

In [1]:
import pandas as pd
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
import numpy as np

import random as rnd

### Сбор мнений экспертов
Опрашивать кого-то мне показалось слишком трудоёмким, поэтому я сделал:  
1. Расставил самостоятельно баллы критериям по важности для меня
2. Задал эти баллы как математическое ожидание `muList`
3. Задал дисперсию `sigmaList`, на глаз, примерно в районе трети от средних значений
4. Сгенерировал 'менения': 10 случайных чисел по нормальному закону распределения

In [2]:
muList = [10, 1, 8, 2, 8, 15]
sigmaList = [3.3,2,8/3,2,3,6]

critetion_measures = []
for mu, sigma in zip(muList,sigmaList):
    critetion_measures.append([
        rnd.normalvariate(mu=mu, sigma=sigma)
        for i in range(10)])

мнениеЭкспертов = pd.DataFrame(data = critetion_measures, index = 
'Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.)'.split(',')
            ).T
df = (мнениеЭкспертов.T/мнениеЭкспертов.sum(axis = 1)).T
df.insert(0,'№ эксперта',range(len(df.T.columns)))

In [3]:
print("Cумма всех весовых коэффициентов," +
      " назначенных одним экспертом для" + 
      " различных критериев, равна единице:")

newDf = df.copy()
newDf['Сумма'] = df.iloc[:,1:].sum(axis = 1)
newDf

Cумма всех весовых коэффициентов, назначенных одним экспертом для различных критериев, равна единице:


,№ эксперта,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.),Сумма
0,0,0.173739,0.000802,0.127430,0.064505,0.188590,0.444933,1.0
1,1,0.176464,0.012704,0.190068,0.155391,0.080816,0.384557,1.0
2,2,0.312073,-0.025018,0.186283,-0.038703,0.204742,0.360623,1.0
3,3,0.181926,0.050501,0.134657,0.055239,0.174750,0.402927,1.0
4,4,0.197114,0.011312,0.205279,0.052536,0.278957,0.254802,1.0
5,5,0.260948,0.055880,0.231109,0.011612,0.185874,0.254576,1.0
6,6,0.136705,0.064087,0.273491,0.008964,0.198432,0.318321,1.0
7,7,0.355760,0.022061,0.165230,-0.044341,0.244378,0.256912,1.0
8,8,0.340604,-0.069040,0.167246,0.076847,0.247832,0.236511,1.0
9,9,0.255157,-0.089014,0.441733,0.051010,0.198390,0.142723,1.0


In [4]:
print("Рассчитаем коэффициент значимости каждого критерия:\n" +
     "\tСРЗНАЧ()\n")

коэфЗначимости = df[df.columns[1:]].mean() 
коэфЗначимостиDF = pd.DataFrame(df[df.columns[1:]].mean(), columns = ['коэф. значимости']).T
newDf = pd.concat([df, коэфЗначимостиDF])
newDf['Сумма'] = newDf.iloc[:,1:].sum(axis = 1)
newDf

Рассчитаем коэффициент значимости каждого критерия:
	СРЗНАЧ()



,№ эксперта,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.),Сумма
0,0.0,0.173739,0.000802,0.127430,0.064505,0.188590,0.444933,1.0
1,1.0,0.176464,0.012704,0.190068,0.155391,0.080816,0.384557,1.0
2,2.0,0.312073,-0.025018,0.186283,-0.038703,0.204742,0.360623,1.0
3,3.0,0.181926,0.050501,0.134657,0.055239,0.174750,0.402927,1.0
4,4.0,0.197114,0.011312,0.205279,0.052536,0.278957,0.254802,1.0
5,5.0,0.260948,0.055880,0.231109,0.011612,0.185874,0.254576,1.0
6,6.0,0.136705,0.064087,0.273491,0.008964,0.198432,0.318321,1.0
7,7.0,0.355760,0.022061,0.165230,-0.044341,0.244378,0.256912,1.0
8,8.0,0.340604,-0.069040,0.167246,0.076847,0.247832,0.236511,1.0
9,9.0,0.255157,-0.089014,0.441733,0.051010,0.198390,0.142723,1.0


In [5]:
print("Для определения согласованности мнений экспертов необходимо " +
      "определить коэффициент вариабельности v, который показывает " +
      "величину разброса экспертных оценок. Он определяется как " +
      "отношение среднеквадр. отклонения к коэф. значимости:\n")

среднеквадрОткл = df[df.columns[1:]].std()

коэфВариабельности = среднеквадрОткл/коэфЗначимости
#print(коэфВариабельности)


среднеквадрОтклDF = pd.DataFrame(среднеквадрОткл, columns = ['среднеквадр. откл']).T
коэфВариабельностиDF = pd.DataFrame(коэфВариабельности, columns = ['коэф. вариабельности']).T

print('\nПри v < 0,2 оценки экспертов можно считать согласованными.',
      "\033[1mВ случае v > 0,2\033[0m целесообразно провести с экспертами",
      "содержательное обсуждение важности оцениваемых параметров,",
      'после чего повторить экспертизу.')

newDf = pd.concat([df, коэфЗначимостиDF, среднеквадрОтклDF, коэфВариабельностиDF])
#newDf['Сумма'] = newDf.iloc[:,1:].sum(axis = 1)
newDf

Для определения согласованности мнений экспертов необходимо определить коэффициент вариабельности v, который показывает величину разброса экспертных оценок. Он определяется как отношение среднеквадр. отклонения к коэф. значимости:


При v < 0,2 оценки экспертов можно считать согласованными. В случае v > 0,2 целесообразно провести с экспертами содержательное обсуждение важности оцениваемых параметров, после чего повторить экспертизу.


,№ эксперта,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.)
0,0.0,0.173739,0.000802,0.127430,0.064505,0.188590,0.444933
1,1.0,0.176464,0.012704,0.190068,0.155391,0.080816,0.384557
2,2.0,0.312073,-0.025018,0.186283,-0.038703,0.204742,0.360623
3,3.0,0.181926,0.050501,0.134657,0.055239,0.174750,0.402927
4,4.0,0.197114,0.011312,0.205279,0.052536,0.278957,0.254802
5,5.0,0.260948,0.055880,0.231109,0.011612,0.185874,0.254576
6,6.0,0.136705,0.064087,0.273491,0.008964,0.198432,0.318321
7,7.0,0.355760,0.022061,0.165230,-0.044341,0.244378,0.256912
8,8.0,0.340604,-0.069040,0.167246,0.076847,0.247832,0.236511
9,9.0,0.255157,-0.089014,0.441733,0.051010,0.198390,0.142723


## 1.2. Метод парных сравнений

Определим число оцениваемых параметров m и число экспертов n. В нашем
случае: m=5; n=8

In [6]:
таблицыЭкспертов = []
for expertNum in df.index:
    столбцы = df.columns
    таблица = pd.DataFrame(
        data = [], 
        columns = df.columns[1:], 
        index = df.columns[1:] )
    таблицыЭкспертов.append(таблица)
    
    for k in range(1, len(df.columns)):
        столбец_k = столбцы[k]
        for j in range(1, len(df.columns)):
            столбец_j = столбцы[j]
            if k - j > -1: pass # обход правого верхнего угла только
            elif df[столбец_k][expertNum] > df[столбец_j][expertNum]:
                таблица.iat[k-1,j-1] = 1
                таблица.iat[j-1,k-1] = 0
            #elif df[столбец_k][expertNum] == df[столбец_j][expertNum]:
            #    таблица.iat[k-1,j-1] = 0.5
            #    таблица.iat[j-1,k-1] = 0.5
            #    # нельзя 0,5, нужно избавиться
            else: 
                таблица.iat[k-1,j-1] = 0
                таблица.iat[j-1,k-1] = 1



In [7]:
таблицыЭкспертов[0]

,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.)
Район,NaN,1,1,1,0,0
количество комнат,0,NaN,0,0,0,0
площадь (м2),0,1,NaN,1,0,0
этаж,0,1,0,NaN,0,0
материал стен,1,1,1,1,NaN,0
стоимость (тыс. руб.),1,1,1,1,1,NaN


In [8]:
таблицыЭкспертов[1]

,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.)
Район,NaN,1,0,1,1,0
количество комнат,0,NaN,0,0,0,0
площадь (м2),1,1,NaN,1,1,0
этаж,0,1,0,NaN,1,0
материал стен,0,1,0,0,NaN,0
стоимость (тыс. руб.),1,1,1,1,1,NaN


In [9]:
таблицыЭкспертов[2]

,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.)
Район,NaN,1,1,1,1,0
количество комнат,0,NaN,0,1,0,0
площадь (м2),0,1,NaN,1,0,0
этаж,0,0,0,NaN,0,0
материал стен,0,1,1,1,NaN,0
стоимость (тыс. руб.),1,1,1,1,1,NaN


In [10]:
таблицыЭкспертов[3]

,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.)
Район,NaN,1,1,1,1,0
количество комнат,0,NaN,0,0,0,0
площадь (м2),0,1,NaN,1,0,0
этаж,0,1,0,NaN,0,0
материал стен,0,1,1,1,NaN,0
стоимость (тыс. руб.),1,1,1,1,1,NaN


In [11]:
таблицыЭкспертов[4]

,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.)
Район,NaN,1,0,1,0,0
количество комнат,0,NaN,0,0,0,0
площадь (м2),1,1,NaN,1,0,0
этаж,0,1,0,NaN,0,0
материал стен,1,1,1,1,NaN,1
стоимость (тыс. руб.),1,1,1,1,0,NaN


In [12]:
таблицыЭкспертов[5]

,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.)
Район,NaN,1,1,1,1,1
количество комнат,0,NaN,0,1,0,0
площадь (м2),0,1,NaN,1,1,0
этаж,0,0,0,NaN,0,0
материал стен,0,1,0,1,NaN,0
стоимость (тыс. руб.),0,1,1,1,1,NaN


In [13]:
таблицыЭкспертов[6]

,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.)
Район,NaN,1,0,1,0,0
количество комнат,0,NaN,0,1,0,0
площадь (м2),1,1,NaN,1,1,0
этаж,0,0,0,NaN,0,0
материал стен,1,1,0,1,NaN,0
стоимость (тыс. руб.),1,1,1,1,1,NaN


In [14]:
таблицыЭкспертов[7]

,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.)
Район,NaN,1,1,1,1,1
количество комнат,0,NaN,0,1,0,0
площадь (м2),0,1,NaN,1,0,0
этаж,0,0,0,NaN,0,0
материал стен,0,1,1,1,NaN,0
стоимость (тыс. руб.),0,1,1,1,1,NaN


In [15]:
результатыЭкспертизы = pd.DataFrame(
        data = [таблицыЭкспертов[i].sum(axis = 1)/таблицыЭкспертов[i].sum(axis = 1).sum() for i in range(len(таблицыЭкспертов))], 
        columns = df.columns[1:], 
        index = [f'Эксперт  {i+1}' for i in range(len(таблицыЭкспертов))])

результатыЭкспертизы = pd.concat([результатыЭкспертизы, pd.DataFrame(результатыЭкспертизы.mean(), columns =  pd.Index(['коэф. значимости',])).T])
результатыЭкспертизы['Сумма'] = результатыЭкспертизы.sum(axis = 1)
результатыЭкспертизы

,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.),Сумма
Эксперт 1,0.200000,0.000000,0.133333,0.066667,0.266667,0.333333,1.0
Эксперт 2,0.200000,0.000000,0.266667,0.133333,0.066667,0.333333,1.0
Эксперт 3,0.266667,0.066667,0.133333,0.000000,0.200000,0.333333,1.0
Эксперт 4,0.266667,0.000000,0.133333,0.066667,0.200000,0.333333,1.0
Эксперт 5,0.133333,0.000000,0.200000,0.066667,0.333333,0.266667,1.0
Эксперт 6,0.333333,0.066667,0.200000,0.000000,0.133333,0.266667,1.0
Эксперт 7,0.133333,0.066667,0.266667,0.000000,0.200000,0.333333,1.0
Эксперт 8,0.333333,0.066667,0.133333,0.000000,0.200000,0.266667,1.0
Эксперт 9,0.333333,0.000000,0.133333,0.066667,0.266667,0.200000,1.0
Эксперт 10,0.266667,0.000000,0.333333,0.066667,0.200000,0.133333,1.0


## 1.3. Метод ранжировок

#### Сначала эксперты располагают критерии в порядке значимости

In [16]:
df

,№ эксперта,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.)
0,0,0.173739,0.000802,0.127430,0.064505,0.188590,0.444933
1,1,0.176464,0.012704,0.190068,0.155391,0.080816,0.384557
2,2,0.312073,-0.025018,0.186283,-0.038703,0.204742,0.360623
3,3,0.181926,0.050501,0.134657,0.055239,0.174750,0.402927
4,4,0.197114,0.011312,0.205279,0.052536,0.278957,0.254802
5,5,0.260948,0.055880,0.231109,0.011612,0.185874,0.254576
6,6,0.136705,0.064087,0.273491,0.008964,0.198432,0.318321
7,7,0.355760,0.022061,0.165230,-0.044341,0.244378,0.256912
8,8,0.340604,-0.069040,0.167246,0.076847,0.247832,0.236511
9,9,0.255157,-0.089014,0.441733,0.051010,0.198390,0.142723


In [17]:
def range_by_priority(sequense): #sequense = df.iloc[0,1:]
    
    sortedSeq = sorted(sequense, reverse = True)
    ranged = []
    # range by priority
    for num in sequense:
        ranged.append(sortedSeq.index(num)+1)

    # decrease indexes
    for i in range(len(ranged)-1,0,-1):
        if i in ranged: continue
        ranged = [i if (_ == i+1) else _ for _ in ranged]
    
    return ranged

In [18]:
ранжированныеКоэф = pd.DataFrame(columns = df.columns[1:],
    index = [f'Эксперт  {i+1}' for i in range(len(df))],
    data = [range_by_priority(df.T[row][1:]) for row in df.T])

ранжированныеКоэф['Сумма'] = ранжированныеКоэф.sum(axis = 1)
ранжированныеКоэф

,Район,количество комнат,площадь (м2),этаж,материал стен,стоимость (тыс. руб.),Сумма
Эксперт 1,3,6,4,5,2,1,21
Эксперт 2,3,6,2,4,5,1,21
Эксперт 3,2,5,4,6,3,1,21
Эксперт 4,2,6,4,5,3,1,21
Эксперт 5,4,6,3,5,1,2,21
Эксперт 6,1,5,3,6,4,2,21
Эксперт 7,4,5,2,6,3,1,21
Эксперт 8,1,5,4,6,3,2,21
Эксперт 9,1,6,4,5,2,3,21
Эксперт 10,2,6,1,5,3,4,21


In [19]:
def set_numbers(series): #sequense_ranged_by_priority
    #series = (ранжированныеКоэф.T[ранжированныеКоэф.T.columns[0]][:-1])
    dct = (list(zip(series.index, series.values)))
    srtd = sorted(dct, key=lambda x: x[1])
    koef = 0
    for i in range(len(srtd)):
        srtd[i] = (srtd[i][0], i+1)

    return dict(srtd)

In [20]:
numbers = dict(set_numbers(ранжированныеКоэф.T[ранжированныеКоэф.T.columns[0]][:-1]))
numbers

{'стоимость (тыс. руб.)': 1,
 'материал стен': 2,
 'Район': 3,
 'площадь (м2)': 4,
 'этаж': 5,
 'количество комнат': 6}

#### нумерованные по значимости критерии

In [21]:
data = \
    [set_numbers(ранжированныеКоэф.T[row][:-1])
     for row in ранжированныеКоэф.T.columns]
нумерованные = pd.DataFrame(data = data, index = ранжированныеКоэф.T.columns)
нумерованные

,стоимость (тыс. руб.),материал стен,Район,площадь (м2),этаж,количество комнат
Эксперт 1,1,2,3,4,5,6
Эксперт 2,1,5,3,2,4,6
Эксперт 3,1,3,2,4,6,5
Эксперт 4,1,3,2,4,5,6
Эксперт 5,2,1,4,3,5,6
Эксперт 6,2,4,1,3,6,5
Эксперт 7,1,3,4,2,6,5
Эксперт 8,2,3,1,4,6,5
Эксперт 9,3,2,1,4,5,6
Эксперт 10,4,3,2,1,5,6


In [22]:
def indexes_by_value(series, search_value):
    return [index for index,value in series.items() if value == search_value]

In [23]:
def set_rank(ranked, numbered): #sequense_..._by_priority
    for value in ranked.values:
        indexes = indexes_by_value(ranked, value)
        sum = 0
        for index in indexes:
            sum += numbered[index]
        newValue = sum / len(indexes)
        for index in indexes:
            numbered[index] = newValue

### я очень устал, что начал задумываться с какой стороны от знака "/" делитель, а с какой - делимое...
# я думаю с этим было бы быстрее, но много кода - меньше понимания
#newDict = {}
#for i in range(len(ranked)):
#    if ranked[i] in newDict.keys():
#        newDict[ranked[i]] += 1
#    else: 
#        newDict[ranked[i]] = 1
#newDict
    
    return numbered

In [24]:
ranked = ранжированныеКоэф.T[ранжированныеКоэф.T.columns[0]][:-1]
numbered = нумерованные.T[нумерованные.T.columns[0]]

#### вычисление рангов критериев

In [25]:
data = [
    set_rank(
        ранжированныеКоэф.T[ранжированныеКоэф.T.columns[i]][:-1],
        нумерованные.T[нумерованные.T.columns[i]]
    ) for i in range(len(нумерованные.T.columns))
    ]

ранги = pd.DataFrame(data = data)
ранги['Сумма']= ранги.sum(axis = 1)
ранги

,стоимость (тыс. руб.),материал стен,Район,площадь (м2),этаж,количество комнат,Сумма
Эксперт 1,1,2,3,4,5,6,21
Эксперт 2,1,5,3,2,4,6,21
Эксперт 3,1,3,2,4,6,5,21
Эксперт 4,1,3,2,4,5,6,21
Эксперт 5,2,1,4,3,5,6,21
Эксперт 6,2,4,1,3,6,5,21
Эксперт 7,1,3,4,2,6,5,21
Эксперт 8,2,3,1,4,6,5,21
Эксперт 9,3,2,1,4,5,6,21
Эксперт 10,4,3,2,1,5,6,21


In [26]:
_ = pd.DataFrame(ранги[ранги.columns[:-1]].sum(), columns = ['сумма рангов']).T
temp = pd.concat([ранги, _])
temp

,стоимость (тыс. руб.),материал стен,Район,площадь (м2),этаж,количество комнат,Сумма
Эксперт 1,1,2,3,4,5,6,21.0
Эксперт 2,1,5,3,2,4,6,21.0
Эксперт 3,1,3,2,4,6,5,21.0
Эксперт 4,1,3,2,4,5,6,21.0
Эксперт 5,2,1,4,3,5,6,21.0
Эксперт 6,2,4,1,3,6,5,21.0
Эксперт 7,1,3,4,2,6,5,21.0
Эксперт 8,2,3,1,4,6,5,21.0
Эксперт 9,3,2,1,4,5,6,21.0
Эксперт 10,4,3,2,1,5,6,21.0


In [27]:
n = len(ранги.T.columns)
m = len(ранги.columns[:-1])
Smean = n * (m + 1) / 2
print(f'Кол-во экспертов = {n},',f'кол-во критериев = {m},',f'Среднее = {Smean}',sep='\n')

Кол-во экспертов = 10,
кол-во критериев = 6,
Среднее = 35.0


In [28]:
# _ = pd.DataFrame(ранги[ранги.columns[:-1]].sum(), columns = ['']).T
d = temp.loc['сумма рангов'][:-1].values - Smean
_ = pd.DataFrame(d, index = temp.columns[:-1], columns = ['di'] ).T
temp = pd.concat([temp, _])
K = (d*d).sum()
temp

,стоимость (тыс. руб.),материал стен,Район,площадь (м2),этаж,количество комнат,Сумма
Эксперт 1,1.0,2.0,3.0,4.0,5.0,6.0,21.0
Эксперт 2,1.0,5.0,3.0,2.0,4.0,6.0,21.0
Эксперт 3,1.0,3.0,2.0,4.0,6.0,5.0,21.0
Эксперт 4,1.0,3.0,2.0,4.0,5.0,6.0,21.0
Эксперт 5,2.0,1.0,4.0,3.0,5.0,6.0,21.0
Эксперт 6,2.0,4.0,1.0,3.0,6.0,5.0,21.0
Эксперт 7,1.0,3.0,4.0,2.0,6.0,5.0,21.0
Эксперт 8,2.0,3.0,1.0,4.0,6.0,5.0,21.0
Эксперт 9,3.0,2.0,1.0,4.0,5.0,6.0,21.0
Эксперт 10,4.0,3.0,2.0,1.0,5.0,6.0,21.0


In [29]:
diInGrade2 = temp.loc['di'].values**2
_ = pd.DataFrame(diInGrade2, index = temp.columns, columns = ['di^2'] ).T
temp = pd.concat([temp, _])
temp

,стоимость (тыс. руб.),материал стен,Район,площадь (м2),этаж,количество комнат,Сумма
Эксперт 1,1.0,2.0,3.0,4.0,5.0,6.0,21.0
Эксперт 2,1.0,5.0,3.0,2.0,4.0,6.0,21.0
Эксперт 3,1.0,3.0,2.0,4.0,6.0,5.0,21.0
Эксперт 4,1.0,3.0,2.0,4.0,5.0,6.0,21.0
Эксперт 5,2.0,1.0,4.0,3.0,5.0,6.0,21.0
Эксперт 6,2.0,4.0,1.0,3.0,6.0,5.0,21.0
Эксперт 7,1.0,3.0,4.0,2.0,6.0,5.0,21.0
Эксперт 8,2.0,3.0,1.0,4.0,6.0,5.0,21.0
Эксперт 9,3.0,2.0,1.0,4.0,5.0,6.0,21.0
Эксперт 10,4.0,3.0,2.0,1.0,5.0,6.0,21.0


In [30]:
K = temp.loc['di^2'].sum()
Kmax = 1/12*n**2*(m**3-m)
Kconcordation = K/Kmax
print(  f'K = sum(di^2) = {K}',
        f'Kmax = 1/12*n^2*(m^3-m) = {Kmax}',
        f'Kконк = K/Kmax = {Kconcordation}',
          (f'{Kconcordation} > 0.4 - мнения экспертов согласованы') 
          if Kconcordation > 0.4 else 
          (f'{Kconcordation} < 0.4 - мнения экспертов несогласованы'),
        sep = '\n')

K = sum(di^2) = 1250.0
Kmax = 1/12*n^2*(m^3-m) = 1749.9999999999998
Kконк = K/Kmax = 0.7142857142857144
0.7142857142857144 > 0.4 - мнения экспертов согласованы


In [31]:
valueKoef = (1-(ранги.iloc[:,:-1]-1)/m)
valueKoef['Сумма'] = valueKoef.sum(axis = 1)
valueKoef

,стоимость (тыс. руб.),материал стен,Район,площадь (м2),этаж,количество комнат,Сумма
Эксперт 1,1.000000,0.833333,0.666667,0.500000,0.333333,0.166667,3.5
Эксперт 2,1.000000,0.333333,0.666667,0.833333,0.500000,0.166667,3.5
Эксперт 3,1.000000,0.666667,0.833333,0.500000,0.166667,0.333333,3.5
Эксперт 4,1.000000,0.666667,0.833333,0.500000,0.333333,0.166667,3.5
Эксперт 5,0.833333,1.000000,0.500000,0.666667,0.333333,0.166667,3.5
Эксперт 6,0.833333,0.500000,1.000000,0.666667,0.166667,0.333333,3.5
Эксперт 7,1.000000,0.666667,0.500000,0.833333,0.166667,0.333333,3.5
Эксперт 8,0.833333,0.666667,1.000000,0.500000,0.166667,0.333333,3.5
Эксперт 9,0.666667,0.833333,1.000000,0.500000,0.333333,0.166667,3.5
Эксперт 10,0.500000,0.666667,0.833333,1.000000,0.333333,0.166667,3.5


In [32]:
normedValueKoef = (valueKoef.T/valueKoef.iloc[:,-1]).T
normedValueKoef

,стоимость (тыс. руб.),материал стен,Район,площадь (м2),этаж,количество комнат,Сумма
Эксперт 1,0.285714,0.238095,0.190476,0.142857,0.095238,0.047619,1.0
Эксперт 2,0.285714,0.095238,0.190476,0.238095,0.142857,0.047619,1.0
Эксперт 3,0.285714,0.190476,0.238095,0.142857,0.047619,0.095238,1.0
Эксперт 4,0.285714,0.190476,0.238095,0.142857,0.095238,0.047619,1.0
Эксперт 5,0.238095,0.285714,0.142857,0.190476,0.095238,0.047619,1.0
Эксперт 6,0.238095,0.142857,0.285714,0.190476,0.047619,0.095238,1.0
Эксперт 7,0.285714,0.190476,0.142857,0.238095,0.047619,0.095238,1.0
Эксперт 8,0.238095,0.190476,0.285714,0.142857,0.047619,0.095238,1.0
Эксперт 9,0.190476,0.238095,0.285714,0.142857,0.095238,0.047619,1.0
Эксперт 10,0.142857,0.190476,0.238095,0.285714,0.095238,0.047619,1.0


In [36]:
meanNormedValueKoef = normedValueKoef.mean(axis = 0)
meanNormedValueKoefDF = pd.DataFrame(meanNormedValueKoef, columns = ['Искомый коэффициент значимости'])
meanNormedValueKoefDF.to_csv('Искомый коэффициент значимости.csv',sep=';')
meanNormedValueKoefDF

,Искомый коэффициент значимости
стоимость (тыс. руб.),0.247619
материал стен,0.195238
Район,0.223810
площадь (м2),0.185714
этаж,0.080952
количество комнат,0.066667
Сумма,1.000000
